In [1]:
# load tokenizer
from transformers import BertTokenizerFast

user_defined_symbols = ['[PAD]', '[UNK]', '[UNK0]','[UNK1]','[UNK2]','[UNK3]','[UNK4]','[UNK5]','[UNK6]','[UNK7]','[UNK8]','[UNK9]', '[CLS]', '[SEP]', '[MASK]', '[BOS]','[EOS]']
unused_token_num = 200
unused_list = ['[unused{}]'.format(n) for n in range(unused_token_num)]
user_defined_symbols = user_defined_symbols + unused_list

tokenizer = BertTokenizerFast(
    vocab_file = './hf_tokenizer_special/vocab.txt',
    max_len = 512,
    do_lower_case=False,
)

special_token_dic = {'additional_special_tokens': user_defined_symbols}
tokenizer.add_special_tokens(special_token_dic)

212

In [2]:
# load bert model
from transformers import BertForMaskedLM, pipeline

model = BertForMaskedLM.from_pretrained('model_output')
nlp_fill = pipeline('fill-mask', top_k=5, model=model, tokenizer=tokenizer)
nlp_fill('오늘 카페에가서 [MASK]를 먹었는데요, 너무 맛있떠라구요.')

Some weights of the model checkpoint at model_output were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[{'score': 0.24728845059871674,
  'token': 7092,
  'token_str': '아메리카노',
  'sequence': '오늘 카페에가서 아메리카노 를 먹었는데요, 너무 맛있떠라구요.'},
 {'score': 0.1023724377155304,
  'token': 6527,
  'token_str': '커피',
  'sequence': '오늘 카페에가서 커피 를 먹었는데요, 너무 맛있떠라구요.'},
 {'score': 0.04726902395486832,
  'token': 8890,
  'token_str': '아아',
  'sequence': '오늘 카페에가서 아아 를 먹었는데요, 너무 맛있떠라구요.'},
 {'score': 0.040487710386514664,
  'token': 6798,
  'token_str': '디저트',
  'sequence': '오늘 카페에가서 디저트 를 먹었는데요, 너무 맛있떠라구요.'},
 {'score': 0.038824670016765594,
  'token': 10955,
  'token_str': '팥빙수',
  'sequence': '오늘 카페에가서 팥빙수 를 먹었는데요, 너무 맛있떠라구요.'}]

In [3]:
from dbConn.mongo_conn import config

conn = config()
col = conn['travel_ai'].blog_contents

test_set = list(col.find({"num_docs": {"$gte": 1, "$lte": 100}}, {"cleaned_content": 1}))
conn.close()
len(test_set)

233658

In [4]:
sentence_set = []
for t in test_set:
    sentence_set.extend(t['cleaned_content'])
del test_set
sentence_set = list(set(sentence_set))
len(sentence_set)

4850590

In [15]:
import re
import random

from konlpy.tag import Mecab
m = Mecab()

test_set = random.sample(sentence_set, 100)
origin_sent, masked_set = [], []
for sentence in test_set:
    nouns = []
    for (word, pos) in m.pos(sentence):
        if len(word) > 1 and pos == 'NNG' or pos == 'NNP':
            nouns.append(word)
    if len(nouns) > 1:
        for trans_noun in random.sample(nouns, 1):
            origin_sent.append(sentence)
            masked_set.append(sentence.replace(trans_noun, '[MASK]'))
del test_set
print(len(origin_sent), len(origin_sent))

87 87


In [16]:
test_num = 33
print(origin_sent[test_num])
print(masked_set[test_num])

대문짝만하게 모노하라고 써있지는 않음카카오맵보고 찾아가는데 어디가 문짝인지 몰라서 허둥허둥안에서 보기엔 좀 웃겼을 듯 논픽션 애깅이들 VMD 굿~ 논픽션 제품을 오프라인에서 볼 수 있는 곳이몇몇군데 있는 것으로 알고 있는데.
[MASK]만하게 모노하라고 써있지는 않음카카오맵보고 찾아가는데 어디가 문짝인지 몰라서 허둥허둥안에서 보기엔 좀 웃겼을 듯 논픽션 애깅이들 VMD 굿~ 논픽션 제품을 오프라인에서 볼 수 있는 곳이몇몇군데 있는 것으로 알고 있는데.


In [17]:
import re
import pandas as pd

df_result = []
for sent, masked_sent in zip(origin_sent, masked_set):
    score, token_str, sequence = [], [], []
    masked_preds = nlp_fill(masked_sent)
    if type(masked_preds[0]) != list:
        for mask_pred in masked_preds:
            if re.search(u'[가-힣]', mask_pred['token_str']):
                score.append(str(mask_pred['score']))
                token_str.append(mask_pred['token_str'])
                sequence.append(mask_pred['sequence'])
        if score:
            df_result.append((sent, masked_sent, '\n'.join(score), '\n'.join(token_str), '\n'.join(sequence)))
    else:
        for masked_pred_list in masked_preds:
            for masked_pred in masked_pred_list:
                if re.search(u'[가-힣]', masked_pred['token_str']):
                    score.append(str(masked_pred['score']))
                    token_str.append(masked_pred['token_str'])
                    sequence.append(masked_pred['sequence'])
        if score:
            df_result.append((sent, masked_sent, '\n'.join(score), '\n'.join(token_str), '\n'.join(sequence)))

In [10]:
df = pd.DataFrame(df_result, columns=['origin_sentence', 'masked_sentence', 'score', 'pred_word', 'pred_sentence'])
df.head()

,origin_sentence,masked_sentence,score,pred_word,pred_sentence
0,와인 가이드가 있어서와알못은잠시 정독중인 척 해보지만E!N!G!L!I!S!H!뭐 대...,와인 가이드가 있어서와알못은잠시 정독중인 척 해보지만E!N!G!L!I!S!H!뭐 대...,0.056007735431194305\n0.050780776888132095\n0....,난리\n말\n메뉴판,와인 가이드가 있어서와알못은잠시 정독중인 척 해보지만E! N! G! L! I! S!...
1,와인 가이드가 있어서와알못은잠시 정독중인 척 해보지만E!N!G!L!I!S!H!뭐 대...,와인 [MASK]가 있어서와알못은잠시 정독중인 척 해보지만E!N!G!L!I!S!H!...,0.5364090800285339\n0.12141339480876923\n0.041...,##바\n바\n리스트\n메뉴\n아틀리에,와인바 가 있어서와알못은잠시 정독중인 척 해보지만E! N! G! L! I! S! H...
2,같이 간 지인은 중간 부분이 살짝 질기긴한데이정도면 괜찮다고 맛있다함뭐 단백질 보충!,같이 간 지인은 중간 부분이 살짝 질기긴한데이정도면 괜찮다고 맛있다함뭐 [MASK]...,0.13839229941368103\n0.06216518580913544\n0.03...,체력\n기력\n탄수화물\n단백질\n##시기,같이 간 지인은 중간 부분이 살짝 질기긴한데이정도면 괜찮다고 맛있다함뭐 체력 보충!...
3,같이 간 지인은 중간 부분이 살짝 질기긴한데이정도면 괜찮다고 맛있다함뭐 단백질 보충!,같이 간 지인은 중간 [MASK]이 살짝 질기긴한데이정도면 괜찮다고 맛있다함뭐 단백...,0.33190852403640747\n0.0823947861790657\n0.066...,##맛\n##에\n맛,같이 간 지인은 중간맛 이 살짝 질기긴한데이정도면 괜찮다고 맛있다함뭐 단백질 보충!...
4,여기서 기억에 남는 커피는다 맛있지만엘카미노가 기억에 오래 남을 블렌딩커피 같네요쓴...,여기서 [MASK]에 남는 커피는다 맛있지만엘카미노가 [MASK]에 오래 남을 블렌...,0.23676477372646332\n0.0807899683713913\n0.080...,기억\n끝\n아쉬움\n마지막\n여운\n커피\n기억\n마지막,[CLS] 여기서 기억 에 남는 커피는다 맛있지만엘카미노가 [MASK] 에 오래 남...


In [11]:
df.to_csv('./masked_word_pred.csv', index=False, encoding='utf-8')

In [12]:
t = '해돋이펜션에서는 2만원만 추가하면 바베큐를 이용할 수 있어요. [MASK]하게 잘 익고 있는거 보이시나요?'

In [13]:
print(t)
nlp_fill(t)

해돋이펜션에서는 2만원만 추가하면 바베큐를 이용할 수 있어요. [MASK]하게 잘 익고 있는거 보이시나요?


[{'score': 0.6487979292869568,
  'token': 9029,
  'token_str': '노릇노릇',
  'sequence': '해돋이펜션에서는 2만원만 추가하면 바베큐를 이용할 수 있어요. 노릇노릇 하게 잘 익고 있는거 보이시나요?'},
 {'score': 0.07009831070899963,
  'token': 8388,
  'token_str': '노릇',
  'sequence': '해돋이펜션에서는 2만원만 추가하면 바베큐를 이용할 수 있어요. 노릇 하게 잘 익고 있는거 보이시나요?'},
 {'score': 0.027685485780239105,
  'token': 9486,
  'token_str': '지글지글',
  'sequence': '해돋이펜션에서는 2만원만 추가하면 바베큐를 이용할 수 있어요. 지글지글 하게 잘 익고 있는거 보이시나요?'},
 {'score': 0.018906276673078537,
  'token': 9049,
  'token_str': '야들야들',
  'sequence': '해돋이펜션에서는 2만원만 추가하면 바베큐를 이용할 수 있어요. 야들야들 하게 잘 익고 있는거 보이시나요?'},
 {'score': 0.012451782822608948,
  'token': 11800,
  'token_str': '꼬들꼬들',
  'sequence': '해돋이펜션에서는 2만원만 추가하면 바베큐를 이용할 수 있어요. 꼬들꼬들 하게 잘 익고 있는거 보이시나요?'}]

In [66]:
print(t)
nlp_fill(t)

정말 [MASK]한 가격에 질 좋은 고기를 맛볼수 있는 이곳!


[{'score': 0.3683983087539673,
  'token': 6856,
  'token_str': '저렴',
  'sequence': '정말 저렴 한 가격에 질 좋은 고기를 맛볼수 있는 이곳!'},
 {'score': 0.16905361413955688,
  'token': 7997,
  'token_str': '착한',
  'sequence': '정말 착한 한 가격에 질 좋은 고기를 맛볼수 있는 이곳!'},
 {'score': 0.05537271872162819,
  'token': 6490,
  'token_str': '정말',
  'sequence': '정말 정말 한 가격에 질 좋은 고기를 맛볼수 있는 이곳!'},
 {'score': 0.03767986595630646,
  'token': 2738,
  'token_str': '이',
  'sequence': '정말 이 한 가격에 질 좋은 고기를 맛볼수 있는 이곳!'},
 {'score': 0.021080244332551956,
  'token': 75,
  'token_str': '~',
  'sequence': '정말 ~ 한 가격에 질 좋은 고기를 맛볼수 있는 이곳!'}]

In [64]:
print(t)
nlp_fill(t)

삼각지에 요즘 핫하다는 내추럴 [MASK] 먹으러 갔다왔어요. 쿰쿰한 맛이 정말 매력적이랍니다.


[{'score': 0.5964204668998718,
  'token': 6747,
  'token_str': '와인',
  'sequence': '삼각지에 요즘 핫하다는 내추럴 와인 먹으러 갔다왔어요. 쿰쿰한 맛이 정말 매력적이랍니다.'},
 {'score': 0.3033031225204468,
  'token': 12286,
  'token_str': '##와인',
  'sequence': '삼각지에 요즘 핫하다는 내추럴와인 먹으러 갔다왔어요. 쿰쿰한 맛이 정말 매력적이랍니다.'},
 {'score': 0.01330279465764761,
  'token': 13029,
  'token_str': '레드와인',
  'sequence': '삼각지에 요즘 핫하다는 내추럴 레드와인 먹으러 갔다왔어요. 쿰쿰한 맛이 정말 매력적이랍니다.'},
 {'score': 0.007578785065561533,
  'token': 11902,
  'token_str': '위스키',
  'sequence': '삼각지에 요즘 핫하다는 내추럴 위스키 먹으러 갔다왔어요. 쿰쿰한 맛이 정말 매력적이랍니다.'},
 {'score': 0.0028995973989367485,
  'token': 9876,
  'token_str': '하이볼',
  'sequence': '삼각지에 요즘 핫하다는 내추럴 하이볼 먹으러 갔다왔어요. 쿰쿰한 맛이 정말 매력적이랍니다.'}]